In [3]:
# from https://github.com/H-arshit/UNET-On-COCO/blob/master/tf_Keras_COCO_UNET.ipynb
# Importing Data From COCO

from pycocotools import coco, cocoeval, _mask
from pycocotools import mask as maskUtils 
import array
import numpy as np
import skimage.io as io
import matplotlib.pyplot as plt
import pylab
import os
pylab.rcParams['figure.figsize'] = (8.0, 10.0)
%matplotlib inline

In [4]:
COCO_DIR = '/home/kehulse/fiftyone/coco-2017'
TRAIN_IMG = COCO_DIR + '/train/data'
VAL_IMG = COCO_DIR + '/validation/data'
HOME_DIR = '/home/kehulse/'

In [5]:
CATEGORY_NAMES=['person']

ANNOTATION_FILE_VAL = '/home/kehulse/fiftyone/coco-2017/raw/instances_val2017.json'
ANNOTATION_FILE_TRAIN = '/home/kehulse/fiftyone/coco-2017/raw/instances_train2017.json'


coco_train = coco.COCO(ANNOTATION_FILE_TRAIN)
catIds_train = coco_train.getCatIds(catNms=CATEGORY_NAMES);
imgIds_train = coco_train.getImgIds(catIds=catIds_train);
imgDict_train = coco_train.loadImgs(imgIds_train)
len(imgIds_train) , len(catIds_train)





coco_val = coco.COCO(ANNOTATION_FILE_VAL)
catIds_val = coco_val.getCatIds(catNms=CATEGORY_NAMES);
imgIds_val = coco_val.getImgIds(catIds=catIds_val);
imgDict_val = coco_val.loadImgs(imgIds_val)
len(imgIds_val) , len(catIds_val)

loading annotations into memory...
Done (t=21.04s)
creating index...
index created!
loading annotations into memory...
Done (t=3.96s)
creating index...
index created!


(2693, 1)

In [6]:
from random import shuffle


shuffle(imgIds_train)
shuffle(imgIds_val)

imgIds_train = imgIds_train[0:6000]
imgIds_val = imgIds_val[0:600]

In [7]:
train_images_person = ["COCO_train2017_{0:012d}.jpg".format(ids) for ids in imgIds_train]
val_images_person = ["COCO_val2017_{0:012d}.jpg".format(ids) for ids in imgIds_val]

In [8]:
print(len(train_images_person) , len(val_images_person))

6000 600


In [9]:
train_images_person = ["COCO_train2017_{0:012d}.jpg".format(ids) for ids in imgIds_train]
print(train_images_person)
del_img_train = set(os.listdir("/home/kehulse/fiftyone/coco-2017/train/data")) - set(train_images_person)
for file_name in del_img_train:
  file_name = "/home/kehulse/fiftyone/coco-2017/train/data" + file_name
  if os.path.exists(file_name):
    os.remove(file_name)

['COCO_train2017_000000543517.jpg', 'COCO_train2017_000000254838.jpg', 'COCO_train2017_000000420400.jpg', 'COCO_train2017_000000118703.jpg', 'COCO_train2017_000000276063.jpg', 'COCO_train2017_000000528621.jpg', 'COCO_train2017_000000136145.jpg', 'COCO_train2017_000000543062.jpg', 'COCO_train2017_000000403474.jpg', 'COCO_train2017_000000353809.jpg', 'COCO_train2017_000000230701.jpg', 'COCO_train2017_000000491346.jpg', 'COCO_train2017_000000521218.jpg', 'COCO_train2017_000000523245.jpg', 'COCO_train2017_000000098881.jpg', 'COCO_train2017_000000554566.jpg', 'COCO_train2017_000000254301.jpg', 'COCO_train2017_000000554015.jpg', 'COCO_train2017_000000073584.jpg', 'COCO_train2017_000000012933.jpg', 'COCO_train2017_000000461715.jpg', 'COCO_train2017_000000364284.jpg', 'COCO_train2017_000000216618.jpg', 'COCO_train2017_000000187111.jpg', 'COCO_train2017_000000527073.jpg', 'COCO_train2017_000000104596.jpg', 'COCO_train2017_000000033645.jpg', 'COCO_train2017_000000336783.jpg', 'COCO_train2017_000

In [10]:
print(len(os.listdir("/home/kehulse/fiftyone/coco-2017/train/data")))

200


In [11]:
val_images_person = ["COCO_val2017_{0:012d}.jpg".format(ids) for ids in imgIds_val]
print(val_images_person)
del_img_val = set(os.listdir(VAL_IMG)) - set(val_images_person)
for file_name in del_img_val:
  file_name = VAL_IMG + file_name
  if os.path.exists(file_name):
    os.remove(file_name)

len(os.listdir(VAL_IMG))

['COCO_val2017_000000169996.jpg', 'COCO_val2017_000000201072.jpg', 'COCO_val2017_000000278973.jpg', 'COCO_val2017_000000240049.jpg', 'COCO_val2017_000000517056.jpg', 'COCO_val2017_000000485480.jpg', 'COCO_val2017_000000197658.jpg', 'COCO_val2017_000000580294.jpg', 'COCO_val2017_000000286422.jpg', 'COCO_val2017_000000314251.jpg', 'COCO_val2017_000000186624.jpg', 'COCO_val2017_000000017959.jpg', 'COCO_val2017_000000172571.jpg', 'COCO_val2017_000000416104.jpg', 'COCO_val2017_000000012576.jpg', 'COCO_val2017_000000130386.jpg', 'COCO_val2017_000000544444.jpg', 'COCO_val2017_000000172946.jpg', 'COCO_val2017_000000162415.jpg', 'COCO_val2017_000000364102.jpg', 'COCO_val2017_000000263403.jpg', 'COCO_val2017_000000361103.jpg', 'COCO_val2017_000000422706.jpg', 'COCO_val2017_000000549390.jpg', 'COCO_val2017_000000021903.jpg', 'COCO_val2017_000000094614.jpg', 'COCO_val2017_000000477805.jpg', 'COCO_val2017_000000078915.jpg', 'COCO_val2017_000000319534.jpg', 'COCO_val2017_000000213033.jpg', 'COCO_val

50

In [12]:
from PIL import Image

In [13]:
!mkdir mask_train_2017

In [14]:
count = 0 

for ID in imgIds_train:

    file_path = "./mask_train_2017/COCO_train2014_{0:012d}.jpg".format(ID)
  
    sampleImgIds = coco_train.getImgIds(imgIds = [ID])
    sampleImgDict = coco_train.loadImgs(sampleImgIds[np.random.randint(0,len(sampleImgIds))])[0]

    annIds = coco_train.getAnnIds(imgIds=sampleImgDict['id'], catIds=catIds_train, iscrowd=0)
    anns = coco_train.loadAnns(annIds)


    mask = coco_train.annToMask(anns[0])
    for i in range(len(anns)):
        mask = mask | coco_train.annToMask(anns[i])
  
    mask = Image.fromarray(mask * 255 , mode = "L")
    mask.save(file_path)
    count = count + 1
    print(count)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060
2061
2062
2063
2064
2065
2066
2067


3507
3508
3509
3510
3511
3512
3513
3514
3515
3516
3517
3518
3519
3520
3521
3522
3523
3524
3525
3526
3527
3528
3529
3530
3531
3532
3533
3534
3535
3536
3537
3538
3539
3540
3541
3542
3543
3544
3545
3546
3547
3548
3549
3550
3551
3552
3553
3554
3555
3556
3557
3558
3559
3560
3561
3562
3563
3564
3565
3566
3567
3568
3569
3570
3571
3572
3573
3574
3575
3576
3577
3578
3579
3580
3581
3582
3583
3584
3585
3586
3587
3588
3589
3590
3591
3592
3593
3594
3595
3596
3597
3598
3599
3600
3601
3602
3603
3604
3605
3606
3607
3608
3609
3610
3611
3612
3613
3614
3615
3616
3617
3618
3619
3620
3621
3622
3623
3624
3625
3626
3627
3628
3629
3630
3631
3632
3633
3634
3635
3636
3637
3638
3639
3640
3641
3642
3643
3644
3645
3646
3647
3648
3649
3650
3651
3652
3653
3654
3655
3656
3657
3658
3659
3660
3661
3662
3663
3664
3665
3666
3667
3668
3669
3670
3671
3672
3673
3674
3675
3676
3677
3678
3679
3680
3681
3682
3683
3684
3685
3686
3687
3688
3689
3690
3691
3692
3693
3694
3695
3696
3697
3698
3699
3700
3701
3702
3703
3704
3705
3706


5159
5160
5161
5162
5163
5164
5165
5166
5167
5168
5169
5170
5171
5172
5173
5174
5175
5176
5177
5178
5179
5180
5181
5182
5183
5184
5185
5186
5187
5188
5189
5190
5191
5192
5193
5194
5195
5196
5197
5198
5199
5200
5201
5202
5203
5204
5205
5206
5207
5208
5209
5210
5211
5212
5213
5214
5215
5216
5217
5218
5219
5220
5221
5222
5223
5224
5225
5226
5227
5228
5229
5230
5231
5232
5233
5234
5235
5236
5237
5238
5239
5240
5241
5242
5243
5244
5245
5246
5247
5248
5249
5250
5251
5252
5253
5254
5255
5256
5257
5258
5259
5260
5261
5262
5263
5264
5265
5266
5267
5268
5269
5270
5271
5272
5273
5274
5275
5276
5277
5278
5279
5280
5281
5282
5283
5284
5285
5286
5287
5288
5289
5290
5291
5292
5293
5294
5295
5296
5297
5298
5299
5300
5301
5302
5303
5304
5305
5306
5307
5308
5309
5310
5311
5312
5313
5314
5315
5316
5317
5318
5319
5320
5321
5322
5323
5324
5325
5326
5327
5328
5329
5330
5331
5332
5333
5334
5335
5336
5337
5338
5339
5340
5341
5342
5343
5344
5345
5346
5347
5348
5349
5350
5351
5352
5353
5354
5355
5356
5357
5358


In [15]:
!mkdir mask_val_2017

In [16]:
count = 0 
for ID in imgIds_val:

    file_path = f"./mask_val_2017/COCO_val2017_{0:012d}.jpg".format(ID)
  
    sampleImgIds = coco_val.getImgIds(imgIds = [ID])
    sampleImgDict = coco_val.loadImgs(sampleImgIds[np.random.randint(0,len(sampleImgIds))])[0]

    annIds = coco_val.getAnnIds(imgIds=sampleImgDict['id'], catIds=catIds_val, iscrowd=0)
    anns = coco_val.loadAnns(annIds)


    mask = coco_val.annToMask(anns[0])
    for i in range(len(anns)):
        mask = mask | coco_val.annToMask(anns[i])
  
    mask = Image.fromarray(mask * 255 , mode = "L")
    mask.save(file_path)
  
    count = count + 1
    print(count)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


# Data Generator

In [17]:
import os
import sys
import random

import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.optimizers import *


seed = 0

random.seed = seed
np.random.seed = seed 

2022-06-05 22:18:28.804431: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/kehulse/.local/lib/python3.9/site-packages/cv2/../../lib64:/usr/local/nvidia/lib64:/usr/local/cuda/lib64
2022-06-05 22:18:28.804478: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [18]:
class DataGen(tf.keras.utils.Sequence):
  
    def __init__(self , path_input , path_mask , batch_size = 8 , image_size = 128):
    
        self.ids = os.listdir(path_input)
        self.path_input = path_input
        self.path_mask = path_mask
        self.batch_size = batch_size
        self.image_size = image_size
        self.on_epoch_end()
  
    def __load__(self , id_name):
    
        image_path = os.path.join(self.path_input , id_name)
        mask_path = os.path.join(self.path_mask , id_name) 
    
        image = cv2.imread(image_path , 1) # 1 specifies RGB format
        image = cv2.resize(image , (self.image_size , self.image_size)) # resizing before inserting to the network
    
        mask = cv2.imread(mask_path , -1)
        mask = cv2.resize(mask , (self.image_size , self.image_size))
        mask = mask.reshape((self.image_size , self.image_size , 1))
      
        #normalize image
        image = image / 255.0
        mask = mask / 255.0
    
        return image , mask
  
    def __getitem__(self , index):
    
        if (index + 1)*self.batch_size > len(self.ids):
            self.batch_size = len(self.ids) - index * self.batch_size
        
        file_batch = self.ids[index * self.batch_size : (index + 1) * self.batch_size]
    
        images = []
        masks = []
    
        for id_name in file_batch : 
      
            _img , _mask = self.__load__(id_name)
            images.append(_img)
            masks.append(_mask)
    
    
        images = np.array(images)
        masks = np.array(masks)
    
    
        return images , masks
  
  
    def on_epoch_end(self):
        pass
  
  
    def __len__(self):
    
        return int(np.ceil(len(self.ids) / float(self.batch_size)))

# UNet

In [19]:
def down_block(
    input_tensor,
    no_filters,
    kernel_size=(3, 3),
    strides=(1, 1),
    padding="same",
    kernel_initializer="he_normal",
    max_pool_window=(2, 2),
    max_pool_stride=(2, 2)
):
    conv = Conv2D(
        filters=no_filters,
        kernel_size=kernel_size,
        strides=strides,
        activation=None,
        padding=padding,
        kernel_initializer=kernel_initializer
    )(input_tensor)

    conv = BatchNormalization(scale=True)(conv)

    conv = Activation("relu")(conv)

    conv = Conv2D(
        filters=no_filters,
        kernel_size=kernel_size,
        strides=strides,
        activation=None,
        padding=padding,
        kernel_initializer=kernel_initializer
    )(conv)

    conv = BatchNormalization(scale=True)(conv)

    # conv for skip connection
    conv = Activation("relu")(conv)

    pool = MaxPooling2D(pool_size=max_pool_window, strides=max_pool_stride)(conv)

    return conv, pool

In [20]:
def bottle_neck(
    input_tensor,
    no_filters,
    kernel_size=(3, 3),
    strides=(1, 1),
    padding="same",
    kernel_initializer="he_normal"
):
    conv = Conv2D(
        filters=no_filters,
        kernel_size=kernel_size,
        strides=strides,
        activation=None,
        padding=padding,
        kernel_initializer=kernel_initializer
    )(input_tensor)

    conv = BatchNormalization(scale=True)(conv)

    conv = Activation("relu")(conv)

    conv = Conv2D(
        filters=no_filters,
        kernel_size=kernel_size,
        strides=strides,
        activation=None,
        padding=padding,
        kernel_initializer=kernel_initializer
    )(conv)

    conv = BatchNormalization(scale=True)(conv)

    conv = Activation("relu")(conv)

    return conv

In [21]:
def up_block(    
    input_tensor,
    no_filters,
    skip_connection, 
    kernel_size=(3, 3),
    strides=(1, 1),
    upsampling_factor = (2,2),
    max_pool_window = (2,2),
    padding="same",
    kernel_initializer="he_normal"):
    
    
    conv = Conv2D(
        filters = no_filters,
        kernel_size= max_pool_window,
        strides = strides,
        activation = None,
        padding = padding,
        kernel_initializer=kernel_initializer
    )(UpSampling2D(size = upsampling_factor)(input_tensor))
    
    conv = BatchNormalization(scale=True)(conv)

    conv = Activation("relu")(conv) 
    
    
    conv = concatenate( [skip_connection , conv]  , axis = -1)
    
    
    conv = Conv2D(
        filters=no_filters,
        kernel_size=kernel_size,
        strides=strides,
        activation=None,
        padding=padding,
        kernel_initializer=kernel_initializer
    )(conv)

    conv = BatchNormalization(scale=True)(conv)

    conv = Activation("relu")(conv)

    conv = Conv2D(
        filters=no_filters,
        kernel_size=kernel_size,
        strides=strides,
        activation=None,
        padding=padding,
        kernel_initializer=kernel_initializer
    )(conv)

    conv = BatchNormalization(scale=True)(conv)

    conv = Activation("relu")(conv)
    
    return conv

In [22]:
def output_block(input_tensor,
    padding="same",
    kernel_initializer="he_normal"
):
    
    conv = Conv2D(
        filters=2,
        kernel_size=(3,3),
        strides=(1,1),
        activation="relu",
        padding=padding,
        kernel_initializer=kernel_initializer
    )(input_tensor)
    
    
    conv = Conv2D(
        filters=1,
        kernel_size=(1,1),
        strides=(1,1),
        activation="sigmoid",
        padding=padding,
        kernel_initializer=kernel_initializer
    )(conv)
    
    
    return conv

In [23]:
def UNet(input_shape = (128,128,3)):
    
    filter_size = [64,128,256,512,1024]
    
    inputs = Input(shape = input_shape)
    
    d1 , p1 = down_block(input_tensor= inputs,
                         no_filters=filter_size[0],
                         kernel_size = (3,3),
                         strides=(1,1),
                         padding="same",
                         kernel_initializer="he_normal",
                         max_pool_window=(2,2),
                         max_pool_stride=(2,2))
    
    
    d2 , p2 = down_block(input_tensor= p1,
                         no_filters=filter_size[1],
                         kernel_size = (3,3),
                         strides=(1,1),
                         padding="same",
                         kernel_initializer="he_normal",
                         max_pool_window=(2,2),
                         max_pool_stride=(2,2))
    
    
    
    d3 , p3 = down_block(input_tensor= p2,
                         no_filters=filter_size[2],
                         kernel_size = (3,3),
                         strides=(1,1),
                         padding="same",
                         kernel_initializer="he_normal",
                         max_pool_window=(2,2),
                         max_pool_stride=(2,2))
    
    
    
    d4 , p4 = down_block(input_tensor= p3,
                         no_filters=filter_size[3],
                         kernel_size = (3,3),
                         strides=(1,1),
                         padding="same",
                         kernel_initializer="he_normal",
                         max_pool_window=(2,2),
                         max_pool_stride=(2,2))
    
    
    b = bottle_neck(input_tensor= p4,
                         no_filters=filter_size[4],
                         kernel_size = (3,3),
                         strides=(1,1),
                         padding="same",
                         kernel_initializer="he_normal")
    
    
    
    u4 = up_block(input_tensor = b,
                  no_filters = filter_size[3],
                  skip_connection = d4,
                  kernel_size=(3, 3),
                  strides=(1, 1),
                  upsampling_factor = (2,2),
                  max_pool_window = (2,2),
                  padding="same",
                  kernel_initializer="he_normal")
    
    u3 = up_block(input_tensor = u4,
                  no_filters = filter_size[2],
                  skip_connection = d3,
                  kernel_size=(3, 3),
                  strides=(1, 1),
                  upsampling_factor = (2,2),
                  max_pool_window = (2,2),
                  padding="same",
                  kernel_initializer="he_normal")
    
    
    u2 = up_block(input_tensor = u3,
                  no_filters = filter_size[1],
                  skip_connection = d2,
                  kernel_size=(3, 3),
                  strides=(1, 1),
                  upsampling_factor = (2,2),
                  max_pool_window = (2,2),
                  padding="same",
                  kernel_initializer="he_normal")
    
    
    u1 = up_block(input_tensor = u2,
                  no_filters = filter_size[0],
                  skip_connection = d1,
                  kernel_size=(3, 3),
                  strides=(1, 1),
                  upsampling_factor = (2,2),
                  max_pool_window = (2,2),
                  padding="same",
                  kernel_initializer="he_normal")
    
    
    
    output = output_block(input_tensor=u1 , 
                         padding = "same",
                         kernel_initializer= "he_normal")
    
    model = Model(inputs = inputs , outputs = output)
    
    
    return model

In [28]:
model = UNet(input_shape = (128,128,3))
model.compile(optimizer = Adam(learning_rate = 1e-4), loss = 'binary_crossentropy', metrics = ['accuracy'])

In [25]:
image_size = 128 
epochs = 10
batch_size = 8

In [26]:
train_gen = DataGen(path_input = TRAIN_IMG , path_mask = './mask_train_2017' , batch_size = batch_size , image_size = image_size)
val_gen = DataGen(path_input =  VAL_IMG, path_mask =  './mask_val_2017', batch_size = batch_size , image_size = image_size)


train_steps =  len(os.listdir(TRAIN_IMG))/batch_size


model.fit(train_gen , validation_data = val_gen , steps_per_epoch = train_steps , epochs=epochs)

[ WARN:0@4.844] global /io/opencv/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('./mask_train_2017/000000012818.jpg'): can't open/read file: check file path/integrity


error: OpenCV(4.5.5) /io/opencv/modules/imgproc/src/resize.cpp:4052: error: (-215:Assertion failed) !ssize.empty() in function 'resize'
